In [1]:
from sympy import Matrix, Rational
import matplotlib.pyplot as plt
import numpy as np

# TERMINOLOGY:
# - nodes: always the fixed points upon which values are set
# - points: variable coordinates typically used to interpolate on


class Interpolator:
    def __init__(self, r: int, p: int, nodal_values: list = None):
        self.r = r
        self.p = p
        self.powers = self.generate_powers(r)
        self.nodes, self.n = self.generate_interp_nodes(p)
        self.M = self.generate_M_matrix()
        if nodal_values:
            self.set_nodal_values(nodal_values)

    def plot_nodes(self):
        plt.figure()
        plt.scatter(self.nodes[:, 0], self.nodes[:, 1])
        plt.grid()
        plt.show()

    def generate_powers(self, r: int) -> list:
        """
        r: int = degree
        """
        return [(j, i) for i in range(0, r + 1) for j in range(0, r + 1) if i + j <= r]

    def generate_interp_nodes(self, p):
        n = (p + 1) * (p + 2) // 2
        nodes = [
            [Rational(j, p), Rational(i, p)]
            for i in range(0, p + 1)
            for j in range(0, p + 1)
            if i + j <= p
        ]
        return np.array(nodes), n

    def print_polynomial(self, sol) -> None:
        poly = str()
        for ii, (i, j) in enumerate(self.powers):
            s = "basis function number : "
            print("\033[1m" + s + "\033[0m\n", i + 1)
            poly += f"+{sol[ii]}x^{i}y^{j}"
        poly = poly[1:].replace("+", " + ").replace("-", " - ")
        print(poly)

    def evaluate_basis(self, points: list = []):
        if points == []:
            points = self.nodes

        result = []
        for power in self.powers:
            result.append(
                list((points[:, 0] ** power[0]) * (points[:, 1] ** power[1]))
            )

        return np.array(result).transpose()

    def interpolate(self, points):
        X = self.evaluate_basis(points)
        B = [np.matmul(self.M, x) for x in X]
        return [np.matmul(self.nodal_values, b) for b in B]

    def generate_coefficients(self, evaluation_matrix, verbose: bool = False):
        coeff = []

        for i in range(self.n):
            b = [Rational(0) for j in range(self.n)]
            b[i] = Rational(1)
            b = Matrix(b)

            res = evaluation_matrix.solve(b)
            coeff.append(res)

            if verbose:
                self.print_polynomial(self.p, res)

        return coeff

    def generate_M_matrix(self):
        A = Matrix(self.evaluate_basis())
        coeffs = self.generate_coefficients(A)
        return np.reshape(np.array(coeffs, dtype=np.float64), (self.n, self.n))
    
    def generate_M_alpha(self, M: Matrix, r: int):
        coeff_dx=[]
        coeff_dy=[]
        for ii in range(self.n):
            temp_x=[]
            temp_y=[]
            for jj,(i,j) in enumerate(self.generate_powers(r)):
                if (i-1)>=0:
                    temp_x.append(Rational(i,1)*M[ii][jj])
                if (j-1)>=0:
                    temp_y.append(Rational(j,1)*M[ii][jj])
            coeff_dx.append(temp_x)
            coeff_dy.append(temp_y)
        return coeff_dx,coeff_dy

    
    def set_nodal_values(self, nodal_values: list):
        self.nodal_values = nodal_values